In [1]:
import pandas as pd

# Get the data from the model

In [ ]:
#TEMPORARY

# Get the historical betting data

In [4]:
bets = pd.read_csv('../data/csv/finalised_sportsbook.csv')

In [9]:
bets.sort_values('datetime_x', ascending=False).head(50)

,Unnamed: 0,event id,market id,description,location,datetime_x,decimal odds,american odds,spread / total,participants.1.participant id,participants.1.source.nickname,participants.1.source.short name,participants.2.participant id,participants.2.source.nickname,participants.2.source.short name
43902,2371,4668184,402,New Orleans@Golden State,San Francisco,2023-03-03T20:25:24+07:00,1.9434,-106,231.0,1152,Warriors,Golden State,1157,Pelicans,New Orleans
43901,2370,4668184,402,New Orleans@Golden State,San Francisco,2023-03-03T20:25:24+07:00,1.9434,-106,231.0,1152,Warriors,Golden State,1157,Pelicans,New Orleans
43900,2372,4668184,83,New Orleans@Golden State,San Francisco,2023-03-03T20:25:24+07:00,1.5988,-167,0.0,1152,Warriors,Golden State,1157,Pelicans,New Orleans
43899,2373,4668184,83,New Orleans@Golden State,San Francisco,2023-03-03T20:25:24+07:00,2.5000,150,0.0,1152,Warriors,Golden State,1157,Pelicans,New Orleans
43898,2355,4668175,83,LA Clippers@Sacramento Kings,Sacramento,2023-03-03T20:07:52+07:00,2.9800,198,0.0,1150,Kings,Sacramento,1151,L.A. Clippers,L.A. Clippers
43897,2354,4668175,83,LA Clippers@Sacramento Kings,Sacramento,2023-03-03T20:07:52+07:00,1.4444,-225,0.0,1150,Kings,Sacramento,1151,L.A. Clippers,L.A. Clippers
43896,2352,4668175,402,LA Clippers@Sacramento Kings,Sacramento,2023-03-03T20:07:52+07:00,1.9434,-106,242.0,1150,Kings,Sacramento,1151,L.A. Clippers,L.A. Clippers
43895,2353,4668175,402,LA Clippers@Sacramento Kings,Sacramento,2023-03-03T20:07:52+07:00,1.9434,-106,242.0,1150,Kings,Sacramento,1151,L.A. Clippers,L.A. Clippers
43892,2343,4668171,83,Utah Jazz@Oklahoma City,Oklahoma City,2023-03-03T20:05:02+07:00,2.0200,102,0.0,1145,Thunder,Oklahoma City,1143,Jazz,Utah
43891,2342,4668171,83,Utah Jazz@Oklahoma City,Oklahoma City,2023-03-03T20:05:02+07:00,1.8850,-113,0.0,1145,Thunder,Oklahoma City,1143,Jazz,Utah


# Compare the data

# Compare the data using selection